In [ ]:
# | default_exp blocks/se

# Imports

In [ ]:
# | export


from functools import wraps

import torch
from torch import nn

from vision_architectures.blocks.cnn import CNNBlock3D, CNNBlockConfig
from vision_architectures.docstrings import populate_docstring
from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.custom_base_model import Field
from vision_architectures.utils.rearrange import rearrange_channels

# Config

In [ ]:
# | export


class SEBlock3DConfig(CNNBlockConfig):
    dim: int = Field(..., description="Number of input channels.")
    r: float = Field(..., description="Reduction ratio for the number of channels in the SE block.")

    kernel_size: int = Field(1, description="Kernel size for the convolutional layers in the SE block.")
    normalization: str = Field("batchnorm3d", description="Normalization layer to use in the SE block.")
    activation: str = Field("silu", description="Activation function to use in the SE block.")

    in_channels: None = Field(None, description="determined by dim and r")
    out_channels: None = Field(None, description="determined by dim and r")

# Architecture

In [ ]:
# | export


class SEBlock3D(nn.Module):
    @populate_docstring
    def __init__(self, config: SEBlock3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        """Initialize an SEBlock3D block. Activation checkpointing level 2.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            **kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__()

        self.config = SEBlock3DConfig.model_validate(config | kwargs)

        dim = self.config.dim
        r = self.config.r

        excitation_dim = int(dim // r)

        self.squeeze = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.excite = nn.Sequential(
            CNNBlock3D(
                self.config.model_dump()
                | {
                    "in_channels": dim,
                    "out_channels": excitation_dim,
                    "kernel_size": 1,
                    "stride": 1,
                    "padding": 0,
                },
                checkpointing_level,
            ),
            CNNBlock3D(
                self.config.model_dump()
                | {
                    "in_channels": excitation_dim,
                    "out_channels": dim,
                    "kernel_size": 1,
                    "stride": 1,
                    "padding": 0,
                    "activation": "sigmoid",
                },
                checkpointing_level,
            ),
        )

        self.checkpointing_level2 = ActivationCheckpointing(2, checkpointing_level)

    @populate_docstring
    def _forward(self, x: torch.Tensor, channels_first: bool = True) -> torch.Tensor:
        """Forward pass of the SEBlock3D block.

        Args:
            x: {INPUT_3D_DOC}
            channels_first: {CHANNELS_FIRST_DOC}

        Returns:
            {OUTPUT_3D_DOC}
        """
        # x: (b, [dim], z, y, x, [dim])

        x = rearrange_channels(x, channels_first, True)
        # Now x is (b, dim, z, y, x)

        p = self.squeeze(x)
        # (b, dim, 1, 1, 1)
        p = self.excite(p)
        # (b, dim, 1, 1, 1)
        x = x * p
        # (b, dim, z, y, x)

        x = rearrange_channels(x, True, channels_first)
        # (b, [dim], z, y, x, [dim])

        return x

    @wraps(_forward)
    def forward(self, *args, **kwargs):
        return self.checkpointing_level2(self._forward, *args, **kwargs)

In [ ]:
test = SEBlock3D(dim=12, r=3, kernel_size=3)
display(test)

sample_input = torch.randn(2, 12, 4, 4, 4)
test(sample_input).shape


SEBlock3D(
  (squeeze): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  (excite): Sequential(
    (0): CNNBlock3D(
      (conv): Conv3d(12, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (norm): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
      (checkpointing_level1): ActivationCheckpointing(enabled=False)
    )
    (1): CNNBlock3D(
      (conv): Conv3d(4, 12, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (norm): BatchNorm3d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Sigmoid()
      (checkpointing_level1): ActivationCheckpointing(enabled=False)
    )
  )
  (checkpointing_level2): ActivationCheckpointing(enabled=False)
)

torch.Size([2, 12, 4, 4, 4])

# nbdev

In [ ]:
!nbdev_export